#  Texture Synthesis
Image synthesis in a maethod to construct a large scale image using a given image to reconstruct it's texture.

In order to do this, we pick a region from the main image according to the specified length. here we pick from index [0, 0]. we could do that randomly too. after that we should find a block which left region of that is similar to righ region of last block. pick this block, similar in shape, and we put this block next to last block so we can get an overlap region. then we should merge these two blocks to get a smooth area between them. some blocks may have overlap with left block and the upper one. here we merge horizontal and vertical sepereately. we repeat this until the end. here we start from top left point, iterate horizontally and vertically, fill the image row by row to reach the bottom right corner.

To do the algorithm above, we define length of each block, length of overlap region and number of blocks we put side by side, horizontally and vertically.

To synthesize the image, on each step we should find the best match according to overlap regions. while filling the first row we just have vartical overlap region, in first column we have to just horizontaly match them, and for the rest of the blocks we should do both. so we have three different mathing problem based on overlap regions.
To find best block similar in overlap regions, we use cv2.templateMatch. we give this method a mask. this mask defines the overlap region. so the cv2 matchtemplate method matches only that region to find the best similar block.

We have three different masks:


![](mask1.png)
![](mask2.png)
![](mask3.png)

After the template matching, we dont pick the best match because we dont want to contrust the exact texture, we just want to reconstruct a similar texture. so we choose k best matches and randomly pick one of them.
To do this, first we faltten the result matrice of templatmatch method, we use np.partition method to have k maximum values at the end of the array. we pick k maximums from that array and use np.where to get the indexes corresponding to max values that we have. and at the end we pick one of them randomly.



Now we have the similar block according to overlap regions, we should put them side by side and merge them. to merging these areas we find a path which these images are more similar on that. so we should solve minimum boundary problem.
To do this, first we get overlap regions, subtract them to get the difference, get square of that and find minimum cut path using algorithm below:

first we create a list of indexes which we are starting from, error of the path and a corresponding list which store the path. for each index on each step we add [index + 1, index, index - 1] to path, if its a new path we store it and update the error. if we store all paths in a min heap and on each step pop from that and iterate over all paths, we always have minimum cut's path first. so on each step if we check the length and see if it has reached the end or not, we will get the optimum path.
After getting the min cut path we create a mask which one side of that path is true and with using this mask and .copyto function we merge images. in more details, one side of the path in mask(which represents overlap region) is true. and we copy the second block to first block where the mask is true.

We implement horizontal and vertical cut seperately. the idea is similar. they are just different in indexing and shapes.